# 基本的な RAG を python SDK で構築する


In [ ]:
# 環境変数の設定
import requests
import os
import openai
import json
import io
import time
import html
import jsonpickle
import re
import base64
import glob

from dotenv import dotenv_values
from abc import ABC, abstractclassmethod # 抽象メソッド
from pypdf import PdfReader, PdfWriter

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    HnswParameters,
    SemanticPrioritizedFields,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
)
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryCaptionResult,
    QueryAnswerResult,
    SemanticErrorMode,
    SemanticErrorReason,
    SemanticSearchResultsType,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorFilterMode,    
)
from azure.storage.blob import BlobServiceClient
from azure.ai.formrecognizer import DocumentAnalysisClient
from tenacity import retry, stop_after_attempt, wait_random_exponential
from openai import AzureOpenAI

In [ ]:
# 環境変数の設定
env_vars = dotenv_values(".env_rag")

AZURE_OPENAI_KEY = env_vars["AZURE_OPENAI_KEY"]
AZURE_OPENAI_ENDPOINT = env_vars["AZURE_OPENAI_ENDPOINT"]
DEPLOYMENT_NAME = env_vars["DEPLOYMENT_NAME"]
EMBEDDING_MODEL = env_vars["EMBEDDING_MODEL"]

AZURE_STORAGE_CONTAINER = env_vars["AZURE_STORAGE_CONTAINER"]
AZURE_BLOB_STORAGE_STRING = env_vars["AZURE_BLOB_STORAGE_STRING"]

AI_SEARCH_ENDPOINT = env_vars["AI_SEARCH_ENDPOINT"]
AI_SEARCH_ADMIN_KEY = env_vars["AI_SEARCH_ADMIN_KEY"]
INDEX_NAME = "index-20250219"
AI_SEAERCH_CRED = AzureKeyCredential(AI_SEARCH_ADMIN_KEY)

DOCUMENT_INTELLIGENCE_KEY = env_vars["DOCUMENT_INTELLIGENCE_KEY"]
DOCUMENT_INTELLIGENCE_ENDPOINT = env_vars["DOCUMENT_INTELLIGENCE_ENDPOINT"]
DOCUMENT_INTELLIGENCE_CREDS: str = AzureKeyCredential(DOCUMENT_INTELLIGENCE_KEY)


## Azure AI Search に インデックス定義を作成する関数
- Field の設定
- セマンティック検索
- ベクトル検索

In [3]:
# Create Index Schema in Azure AI Search

def create_search_index(index_name):
    index_client = SearchIndexClient(endpoint=AI_SEARCH_ENDPOINT, credential=AI_SEAERCH_CRED)
    if index_name not in index_client.list_index_names():            
        index = SearchIndex(
            name=index_name,
            fields=[
                SimpleField(name="id", type="Edm.String", key=True),
                SearchableField(
                    name="content",
                    type="Edm.String",
                    analyzer_name="ja.lucene"
                ),
                SearchField(
                    name="embedding",
                    type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    hidden=False,
                    searchable=True,
                    filterable=False,
                    sortable=False,
                    facetable=False,
                    vector_search_dimensions=1536,
                    vector_search_configuration="default",
                ),
                SimpleField(
                    name="category",
                    type="Edm.String",
                    filterable=True,
                    facetable=True
                ),
                SimpleField(
                    name="sourcepage",
                    type="Edm.String",
                    filterable=True,
                    facetable=True,
                ),
                SimpleField(
                    name="sourcefile",
                    type="Edm.String",
                    filterable=True,
                    facetable=True,
                ),
                SimpleField(
                    name="url",
                    type="Edm.String",
                    filterable=True,
                    facetable=True,
                ),
            ],
            semantic_settings=SemanticConfiguration(
                configurations=[
                    SemanticConfiguration(
                        name="default",
                        prioritized_fields=SemanticPrioritizedFields(
                            title_field=None,
                            prioritized_content_fields=[
                                SemanticField(field_name="content") 
                            ],
                        ),
                    )
                ]
            ),
            # hnsw, m値, cos類似度
            vector_search=VectorSearch(
                algorithm_configurations=[
                    VectorSearchAlgorithmConfiguration(
                        name="default",
                        kind="hnsw",
                        hnsw_parameters=HnswParameters(
                            m = 4,
                            ef_construction = 400,
                            ef_search = 500,
                            metric="cosine"
                        ),
                    )
                ]
            ),
        )
        print(f"Creating {index_name} search index")
        result = index_client.create_or_update_index(index) 
        print(f' {result.name} created')
    else:
        print(f"Search index {index_name} already exists")

def remove_from_index(filename):
    print(f"Removing sections from '{filename or '<all>'}' from search index '{INDEX_NAME}'")
    search_client = SearchClient(endpoint=AI_SEARCH_ENDPOINT,
                                    index_name=INDEX_NAME,
                                    credential=AI_SEAERCH_CRED)
    while True:
        filter = None if filename is None else f"sourcefile eq '{os.path.basename(filename)}'"
        r = search_client.search("", filter=filter, top=1000, include_total_count=True)
        if r.get_count() == 0:
            break
        r = search_client.delete_documents(documents=[{ "id": d["id"] } for d in r])
        print(f"\tRemoved {len(r)} sections from index")
        # It can take a few seconds for search results to reflect changes, so wait a bit
        time.sleep(2)

## PDF をページごとに分割し、Blob Storage にアップロードする関数

#### PDFの場合は PDFReader を使用

In [4]:
# Split in pages and upload to Blob Storage

def blob_name_from_file_page(filename, page = 0):
    if os.path.splitext(filename)[1].lower() == ".pdf":
        #　ファイル名に空白があったらここ変えればよかったんだな
        return os.path.splitext(os.path.basename(filename))[0] + f"-{page}" + ".pdf"
    else:
        return os.path.basename(filename)

def upload_blobs(filename):
    blob_service_client = BlobServiceClient.from_connection_string(AZURE_BLOB_STORAGE_STRING)
    blob_container = blob_service_client.get_container_client(AZURE_STORAGE_CONTAINER)
    if not blob_container.exists():
        blob_container.create_container()

    # ファイルが PDF の場合、ページに分割し、各ページを個別の Blob としてアップロードする。
    if os.path.splitext(filename)[1].lower() == ".pdf":
        reader = PdfReader(filename)
        pages = reader.pages
        for i in range(len(pages)):
            blob_name = blob_name_from_file_page(filename, i)
            
            f = io.BytesIO()
            writer = PdfWriter()
            writer.add_page(pages[i])
            writer.write(f)
            f.seek(0)
            blob_container.upload_blob(blob_name, f, overwrite=True)
    else:
        blob_name = blob_name_from_file_page(filename)
        # PDF以外は分割せずにUpload
        with open(filename,"rb") as data:
            blob_container.upload_blob(blob_name, data, overwrite=True)

#### Document Intelligence を用いたテキスト抽出
- 表は html に変換して埋め込み

In [5]:
# Use Document Intelligence to OCR texts

def table_to_html(table):
    table_html = "<table>"

    # 各行のセルに対して、列インデックスに基づいてソート
    rows = [sorted([cell for cell in table.cells if cell.row_index == i], key=lambda cell: cell.column_index) for i in range(table.row_count)]
    for row_cells in rows:
        table_html += "<tr>"
        for cell in row_cells:
            tag = "th" if (cell.kind == "columnHeader" or cell.kind == "rowHeader") else "td"
            cell_spans = ""
            if cell.column_span > 1: cell_spans += f" colSpan={cell.column_span}"
            if cell.row_span > 1: cell_spans += f" rowSpan={cell.row_span}"
            table_html += f"<{tag}{cell_spans}>{html.escape(cell.content)}</{tag}>"
        table_html +="</tr>"
    table_html += "</table>"
    return table_html

def get_document_text(filename):
    offset = 0
    page_map = []

    print(f"Extracting text from '{filename}' using Azure AI Document Intelligence")
    document_intelligence_client = DocumentAnalysisClient(
        endpoint=DOCUMENT_INTELLIGENCE_ENDPOINT,
        credential=DOCUMENT_INTELLIGENCE_CREDS, 
        headers={"x-ms-useragent": "azure-search-chat-demo/1.0.0"} # user-agent: 識別子
        )
    with open(filename, "rb") as f:
        # text 抽出
        poller = document_intelligence_client.begin_analyze_document("prebuilt-layout", document = f)
    document_intelligence_results = poller.result()
    
    #　Embed table html in page texts
    for page_num, page in enumerate(document_intelligence_results.pages):
        tables_on_page = [table for table in document_intelligence_results.tables if table.bounding_regions[0].page_number == page_num + 1]

        # mark all positions of the table spans in the page
        page_offset = page.spans[0].offset
        page_length = page.spans[0].length
        table_chars = [-1]*page_length
        for table_id, table in enumerate(tables_on_page):
            for span in table.spans:
                # replace all table spans with "table_id" in table_chars array
                for i in range(span.length):
                    idx = span.offset - page_offset + i
                    if idx >=0 and idx < page_length:
                        table_chars[idx] = table_id

        # build page text by replacing characters in table spans with table html, using table_to_html function
        page_text = ""
        added_tables = set()
        for idx, table_id in enumerate(table_chars):
            if table_id == -1:
                page_text += document_intelligence_results.content[page_offset + idx]
            elif table_id not in added_tables:
                page_text += table_to_html(tables_on_page[table_id])
                added_tables.add(table_id)

        page_text += " "
        page_map.append((page_num, offset, page_text))
        offset += len(page_text)

    return page_map

## Embeddings を計算する関数
- api_version を 2024-02-01 に変更

In [6]:
aoai_client = AzureOpenAI(
  api_key = AZURE_OPENAI_KEY,  
  api_version = "2024-02-01",
  azure_endpoint = AZURE_OPENAI_ENDPOINT
)

# Embeddings API のコールにリトライを設定することで Rate limit に対処
def before_retry_sleep(retry_state):
    print("Rate limited on the OpenAI embeddings API, sleeping before retrying...")

@retry(wait=wait_random_exponential(min=15, max=60), stop=stop_after_attempt(15), before_sleep=before_retry_sleep)
def compute_embedding(text):
    return aoai_client.embeddings.create(input = [text], model=EMBEDDING_MODEL).data[0].embedding

#### チャンキング
- 句読点で区切り、オーバーラップも指定

In [7]:
MAX_SECTION_LENGTH = 1000
SENTENCE_SEARCH_LIMIT = 100
SECTION_OVERLAP = 100

def split_text(page_map, filename):
    SENTENCE_ENDINGS = [".", "!", "?","。"]
    WORDS_BREAKS = [",", ";", ":", " ", "(", ")", "[", "]", "{", "}","、", "\t", "\n"]
    print(f"Splitting '{filename}' into sections")

    def find_page(offset):
        num_pages = len(page_map)
        for i in range(num_pages - 1):
            if offset >= page_map[i][1] and offset < page_map[i + 1][1]:
                return i
        return num_pages - 1

    all_text = "".join(p[2] for p in page_map)
    length = len(all_text)
    start = 0
    end = length
    while start + SECTION_OVERLAP < length: # 全テキストがOverlapより大きい場合
        last_word = -1
        end = start + MAX_SECTION_LENGTH

        if end > length: # 1000文字以内に、全テキストが収まる場合
            end = length
        else:
            # Try to find the end of the sentence
            while end < length and (end - start - MAX_SECTION_LENGTH) < SENTENCE_SEARCH_LIMIT and all_text[end] not in SENTENCE_ENDINGS:
                if all_text[end] in WORDS_BREAKS:
                    last_word = end
                end += 1
            if end < length and all_text[end] not in SENTENCE_ENDINGS and last_word > 0: # 文の終わりが見つかった場合
                end = last_word # Fall back to at least keeping a whole word
        if end < length: # 終了位置が見つからなけらば 1000文字から1字ずつ増やす
            end += 1

        # Try to find the start of the sentence or at least a whole word boundary
        last_word = -1
        while start > 0 and start > end - MAX_SECTION_LENGTH - 2 * SENTENCE_SEARCH_LIMIT and all_text[start] not in SENTENCE_ENDINGS:
            if all_text[start] in WORDS_BREAKS:
                last_word = start
            start -= 1
        if all_text[start] not in SENTENCE_ENDINGS and last_word > 0:
            start = last_word
        if start > 0:
            start += 1

        section_text = all_text[start:end]
        yield (section_text, find_page(start))

        last_table_start = section_text.rfind("<table")
        if (last_table_start > 2 * SENTENCE_SEARCH_LIMIT and last_table_start > section_text.rfind("</table")):
            # If the section ends with an unclosed table, we need to start the next section with the table.
            # If table starts inside SENTENCE_SEARCH_LIMIT, we ignore it, as that will cause an infinite loop for tables longer than MAX_SECTION_LENGTH
            # If last table starts inside SECTION_OVERLAP, keep overlapping
            print(f"Section ends with unclosed table, starting next section with the table at page {find_page(start)} offset {start} table start {last_table_start}")
            start = min(end - SECTION_OVERLAP, start + last_table_start)
        else:
            start = end - SECTION_OVERLAP

    if start + SECTION_OVERLAP < end:
        yield (all_text[start:end], find_page(start))

#### Index に登録するデータのマッピング

In [8]:
def filename_to_id(filename):
    filename_ascii = re.sub("[^0-9a-zA-Z_-]", "_", filename)
    filename_hash = base64.b16encode(filename.encode('utf-8')).decode('ascii')
    return f"file-{filename_ascii}-{filename_hash}"

def create_sections(filename, page_map, use_vectors, category):
    file_id = filename_to_id(filename)
    for i, (content, pagenum) in enumerate(split_text(page_map, filename)):
        section = {
            "id": f"{file_id}-page-{i}",
            "content": content,
            "category": category,
            "sourcepage": blob_name_from_file_page(filename, pagenum),
            "sourcefile": filename,
            "url": filename, # 要実装、webサイトのadd your data は html でBlobに保存して、metadata_storage_sas_token でアクセス
            #"metadata": json.dumps({"page": pagenum, "sourcepage": blob_name_from_file_page(filename, pagenum)})
        }
        
        section["embedding"] = compute_embedding(content) # Embedding 計算結果
        yield section

#### チャンクをインデックス化

In [9]:
def index_sections(filename, sections):
    search_client = SearchClient(
        endpoint=AI_SEARCH_ENDPOINT, index_name=INDEX_NAME, credential=AI_SEAERCH_CRED
    )
    i = 0
    batch = []
    for s in sections:
        batch.append(s)
        i += 1
        if i % 1000 == 0:
            results = search_client.upload_documents(documents=batch)
            succeeded = sum([1 for r in results if r.succeeded])
            print(f"\tIndexed {len(results)} sections, {succeeded} succeeded")
            batch = []

    if len(batch) > 0:
        results = search_client.upload_documents(documents=batch)
        succeeded = sum([1 for r in results if r.succeeded])
        print(f"\tIndexed {len(results)} sections, {succeeded} succeeded")

#### 実行部分


In [10]:
print("Create Search Index...")
create_search_index(INDEX_NAME)
print("Processing files...")

path_pattern = "./sampledata/text/*.pdf"
for filename in glob.glob(path_pattern):
    print(f"Processing '{filename}'")
    try:
        upload_blobs(filename)
        remove_from_index(filename)
        page_map = get_document_text(filename)
        category = os.path.basename(os.path.dirname(filename)) # 親ディレクトリ名
        sections = create_sections(
            os.path.basename(filename), page_map, False, category
        )
        index_sections(os.path.basename(filename), sections)

    except Exception as e:
        print(f"\tGot an error while reading {filename} -> {e} --> skipping file")

Create Search Index...
Search index index-0321 already exists
Processing files...
Processing './sampledata/text\Contoso_Estate_Support_Guide.pdf'
Removing sections from './sampledata/text\Contoso_Estate_Support_Guide.pdf' from search index 'index-0321'
	Removed 6 sections from index
Extracting text from './sampledata/text\Contoso_Estate_Support_Guide.pdf' using Azure AI Document Intelligence
Splitting 'Contoso_Estate_Support_Guide.pdf' into sections
	Indexed 6 sections, 6 succeeded
Processing './sampledata/text\model-work-regulations.pdf'
Removing sections from './sampledata/text\model-work-regulations.pdf' from search index 'index-0321'
	Removed 95 sections from index
Extracting text from './sampledata/text\model-work-regulations.pdf' using Azure AI Document Intelligence
Splitting 'model-work-regulations.pdf' into sections
Section ends with unclosed table, starting next section with the table at page 0 offset 0 table start 794
Section ends with unclosed table, starting next section wi

#### 上記の手順で、ローカルPDF をBlob Storage にアップロードし、インデックス作成が完了します

#### セマンティックハイブリッド検索を行う

In [23]:
user_query = "賃貸物件の予約方法について教えてください。"

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# タイトルフィールドとコンテンツフィールドのEmbeddingsを生成する関数
def embed_user_query(text, model = EMBEDDING_MODEL):
    return aoai_client.embeddings.create(input = [text], model=model).data[0].embedding

vector_query = VectorizedQuery(vector=embed_user_query(user_query), k_nearest_neighbors=50, fields="embedding")

search_client = SearchClient(
    endpoint=AI_SEARCH_ENDPOINT, index_name=INDEX_NAME, credential=AI_SEAERCH_CRED
)

retrieved_docs = search_client.search(
    search_text = user_query,
    vector_queries= [vector_query],
    top=3,
    highlight_fields="content-3",
    select=["sourcepage","content","category"],
    query_type=QueryType.SEMANTIC, 
    semantic_configuration_name='default', 
    query_caption=QueryCaptionType.EXTRACTIVE, 
    query_answer=QueryAnswerType.EXTRACTIVE,
)

semantic_answers = retrieved_docs.get_answers()
print('semantic answer = ',semantic_answers)
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")
    
for doc in retrieved_docs:
    # print(f"Source: {doc['sourcepage']}")  
    # print(f"Score: {doc['@search.score']}")  
    #print(f"Content: {doc['content']}")  
    # print(f"Category: {doc['category']}\n")
    
    captions = doc["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

# 改行や括弧を変換
def nonewlines(s: str) -> str:
    return s.replace('\n', ' ').replace('\r', ' ').replace('[', '【').replace(']', '】')

results =[" SOURCE:" + doc['sourcepage'] + ": " + nonewlines(doc['content']) for doc in retrieved_docs]
print(results)

semantic answer =  []
Caption: 検索·予約方法は以下の通りです。 賃貸物件の検索と予約方法をご紹介します: 1. 賃貸物件を検索する: - 目的地、チェックイン日、チェックアウト日、宿泊人数を入力します。 - 価格帯、物件タイプ、アメニティなどのフィルターを適用して選択肢を絞り込む。 - リストを閲覧して、滞在に最適な場所を見つけましょう。 2. リスティングの詳細を見る: - リスティングをクリックすると、写真、物件概要、レビュー、ホスト情報などの詳 細情報が表示されます、 およびホスト情報を含む詳細情報を表示します。 . 

Caption: 方法は以下の通りです: 1. ログインする: - Contoso<em> Real Estate</em> アカウントにログインします。 2. あなたの予約」に進みます: - プロフィール画像をクリックし、"予約状況 "に移動します。 3. 問題のある予約を選択します: - 問題のあるリスティングに関連付けられている予約を見つけてクリックします。 4. 問題を報告する: - 問題を報告する」ボタンをクリックします。 - リスティングで発生した問題を詳しく説明してください。 5. レポートを提出する: - 必要な情報を提供したら、レポートを送信してください。 当社のチームが問題を調査し、適切な処置を講じます。 5.

Caption: ここではその方法を説明します: 1. ログインする: - Contoso<em> Real Estate</em> アカウントにログインします。 2. あなたの予約」に移動します: - プロフィール画像をクリックし、"予約状況"に移動します。 3. キャンセルする予約を選択します: - キャンセルしたい予約を見つけてクリックします。 4. 予約をキャンセルする: - 予約をキャンセルする」ボタンをクリックします。 - キャンセルポリシーを確認し、手数料の可能性を理解する。 - キャンセルを確認します。 5. キャンセル料: - ホストのキャンセルポリシーによっては、キャンセル料が発生する場合がありま す。 キャンセル料 キャンセルの手続き中に明確に表示されます。 6.

[]


In [16]:
query_generation_prompt = """
以下は、過去の会話の履歴と、Contoso Estate という会社の文書を検索して回答する必要のあるユーザーからの新しい質問です。
会話と新しい質問に基づいて、検索クエリを作成してください。
検索クエリには、引用されたファイルや文書の名前（例:info.txtやdoc.pdf)を含めないでください。
検索クエリには、括弧 []または<<>>内のテキストを含めないでください。
検索クエリを生成できない場合は、数字 0 だけを返してください。
"""

prompt = [{'role': 'system', 'content': query_generation_prompt}]

# Few-shot Samples
query_prompt_few_shots = [
    {'role' : 'user', 'content' : '予約のキャンセル方法についての手順をおしえてください' },
    {'role' : 'assistant', 'content' : '予約 キャンセル 方法' },
    {'role' : 'user', 'content' : '問い合わせ先の情報を教えてください' },
    {'role' : 'assistant', 'content' : 'お問い合わせ 連絡先' }
]

for shot in query_prompt_few_shots:
    prompt.append({'role': shot.get('role'), 'content': shot.get('content')})

prompt.append({'role': 'user', 'content': user_query})

print(prompt)

chat_completion = aoai_client.chat.completions.create(
    messages=prompt,
    model=DEPLOYMENT_NAME,
    temperature=0.0,
    max_tokens=100,
    n=1)

query_text = chat_completion.choices[0].message.content
print('生成されたクエリ：',query_text)

[{'role': 'system', 'content': '\n以下は、過去の会話の履歴と、Contoso Estate という会社の文書を検索して回答する必要のあるユーザーからの新しい質問です。\n会話と新しい質問に基づいて、検索クエリを作成してください。\n検索クエリには、引用されたファイルや文書の名前（例:info.txtやdoc.pdf)を含めないでください。\n検索クエリには、括弧 []または<<>>内のテキストを含めないでください。\n検索クエリを生成できない場合は、数字 0 だけを返してください。\n'}, {'role': 'user', 'content': '予約のキャンセル方法についての手順をおしえてください'}, {'role': 'assistant', 'content': '予約 キャンセル 方法'}, {'role': 'user', 'content': '問い合わせ先の情報を教えてください'}, {'role': 'assistant', 'content': 'お問い合わせ 連絡先'}, {'role': 'user', 'content': '賃貸物件の予約方法について教えてください。'}]
生成されたクエリ： 賃貸物件 予約 方法


In [20]:
system_prompt = """
あなたは Contoso Estate のカスタマーサポートとして、ユーザーからの問いに丁寧に答えます。
出典から答えが推測できない場合は「わからない」と答えること。
回答は日本語で生成しなさい。

# 制限事項
- 回答で使用される事実は、その出典を回答の最後に含まなければならない。
- 出典を参照するには角括弧を使用する。例えば、[info1.txt]。出典を結合せず、各出典を個別に記載すること。例：[info1.txt][info2.pdf]。
"""

messages = [{'role': 'system', 'content': system_prompt}]

In [21]:
# Context from Azure AI Search
context = "\n".join(results)
messages.append({'role': 'user', 'content': user_query + "\n\n" + context}) 

chat_coroutine = aoai_client.chat.completions.create(
    model=DEPLOYMENT_NAME,
    messages=messages,
    temperature=0.0,
    max_tokens=1024,
    n=1,
    stream=False
)

print(chat_coroutine.choices[0].message.content)

Contoso Estateの賃貸物件の予約方法については、以下の手順に従って予約することができます。

1. Contoso Estateのウェブサイトにアクセスします。
2. メニューから「賃貸物件」を選択します。
3. 検索フォームに希望する条件（地域、家賃、間取りなど）を入力し、検索ボタンをクリックします。
4. 検索結果から予約したい物件を選択します。
5. 物件の詳細ページで、予約ボタンをクリックします。
6. 予約フォームに必要事項（氏名、連絡先、予約期間など）を入力し、送信ボタンをクリックします。
7. Contoso Estateの担当者から予約の確認や支払い方法についての連絡があります。

以上がContoso Estateの賃貸物件の予約方法です。詳細や特定の物件に関する情報は、Contoso Estateのウェブサイトをご確認ください。

[info1.txt]


In [8]:
import openai
from openai import AzureOpenAI

key = "07a02365e5fd45bc97fa4646b152d840"
base = "https://aoai-japaneast-jk.openai.azure.com/"

client = AzureOpenAI(
    api_key=key,  
    api_version="2024-02-01",
    azure_endpoint=base
    )

In [9]:
embedding_model = "text-embedding-ada-002-jk"
def get_embedding(text):
    result = client.embeddings.create(
      model=embedding_model,
      input=text
    )
    result = result.data[0].embedding
    return result

test = get_embedding("Testing embedding function")
print(test[0:5])

[-0.028185436502099037, 0.0029579917900264263, -0.01179675292223692, -0.0010546649573370814, 0.0029527097940444946]
